# FNN


In [1]:
import numpy as np
import torch

# Setando sementes
np.random.seed(42)
torch.manual_seed(42)

# Configurando tipo das variáveis
dtype = torch.float32
torch.set_default_dtype(dtype)

# Definindo nome do modelo
model_name = "FNN"


## Lendo dados e preparando tensores


In [2]:
import pandas as pd

df = pd.read_csv("../export/dados.csv", index_col=0)

ciclos = df.index.values
t_feed = df["t_feed"].values
t_rinse = df["t_rinse"].values
t_blow = df["t_blow"].values
t_purge = df["t_purge"].values

purity_H2 = df["purity_H2"].values
H2_CO_ratio = df["H2_CO_ratio"].values
purity_CO2 = df["purity_CO2"].values
recovery_CO2 = df["recovery_CO2"].values
productivity = df["productivity"].values


In [3]:
y = np.array(
    [
        purity_H2,
        H2_CO_ratio,
        purity_CO2,
        recovery_CO2,
        productivity,
    ]
).T
u = np.array([t_feed, t_rinse, t_blow, t_purge]).T


In [4]:
from lib.models import DataInfo

# Concatena entradas (y_k-1, u_k-1, u_k)
X = np.hstack([y[:-1], u[:-1], u[1:]])  # (N-1, 13)
Y = y[1:]  # (N-1, 5)

# Converte para tensores
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

dataInfo: DataInfo = {
    "in_min": torch.min(X, dim=0).values,
    "in_max": torch.max(X, dim=0).values,
    "out_min": torch.min(Y, dim=0).values,
    "out_max": torch.max(Y, dim=0).values,
}

# Divide em treino e validação
N = len(X)
N_train = int(0.8 * N)
N_val = N - N_train

X_train = X[:N_train]
Y_train = Y[:N_train]

X_val = X[N_train:]
Y_val = Y[N_train:]


## Criando e Treinando a Rede Neural


In [5]:
from lib.models import FNN

input_size = X.shape[1]
hidden_size = 16
output_size = Y.shape[1]

model = FNN(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    dataInfo=dataInfo,
)


In [6]:
import os

from lib.plot import plot_loss

MODEL_PATH = f"../export/{model_name}.pt"

if os.path.exists(MODEL_PATH):
    print("📦 Modelo encontrado. Carregando...")
    model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))
    model.eval()
else:
    print("🚀 Modelo não encontrado. Treinando agora...")

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    history, best_epoch = model.fit(
        X_train=X_train,
        Y_train=Y_train,
        X_val=X_val,
        Y_val=Y_val,
        optimizer=optimizer,
        epochs=10000,
    )

    plot_loss(
        history["train_loss"][: best_epoch + 1],
        history["val_loss"][: best_epoch + 1],
        model_name,
    )

    # Salvar modelo após treino
    torch.save(model.state_dict(), MODEL_PATH)
    print(f"💾 Modelo salvo em {MODEL_PATH}")


📦 Modelo encontrado. Carregando...


## Simulando


In [7]:
from lib.plot import plot_comparison
from lib.utils import benchmark


def infer_FNN(model, y, u):
    y_nn = np.zeros_like(y)
    y_nn[0] = y[0]  # inicializa com o primeiro valor

    model.eval()
    for k in range(1, len(y)):
        phi_k = np.concatenate([y_nn[k - 1], u[k - 1], u[k]])
        phi_k = torch.tensor(phi_k, dtype=torch.float32)
        y_nn[k] = model(phi_k).detach().numpy()
    return y_nn


y_nn = infer_FNN(model, y, u)
plot_comparison(ciclos, y, y_nn, model_name)

benchmark(infer_FNN, model_name, model, y, u)


Benchmark file already exists. Skipping benchmark.
